In [2]:
import torch

from torch import nn, optim

import torch.nn.functional as F

from torch.autograd import Variable

from torch.utils.data import DataLoader

from torchvision import transforms

from torchvision import datasets

#from logger import Logger

In [3]:
# 定义超参数

batch_size = 128

learning_rate = 1e-2

num_epoches = 20



In [4]:
def to_np(x):

    return x.cpu().data.numpy()

In [5]:
# 下载训练集 MNIST 手写数字训练集

train_dataset = datasets.MNIST(

    root='./data', train=True, transform=transforms.ToTensor(), download=True)



test_dataset = datasets.MNIST(

    root='./data', train=False, transform=transforms.ToTensor())



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [6]:
# 定义 Convolution Network 模型

class Cnn(nn.Module):

    def __init__(self, in_dim, n_class):

        super(Cnn, self).__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),

            nn.ReLU(True),

            nn.MaxPool2d(2, 2),

            nn.Conv2d(6, 16, 5, stride=1, padding=0),

            nn.ReLU(True), nn.MaxPool2d(2, 2))



        self.fc = nn.Sequential(

            nn.Linear(400, 120), nn.Linear(120, 84), nn.Linear(84, n_class))



    def forward(self, x):

        out = self.conv(x)

        out = out.view(out.size(0), -1)

        out = self.fc(out)

        return out



In [8]:
model = Cnn(1, 10)  # 图片大小是28x28

use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速

if use_gpu:

    model = model.cuda()

# 定义loss和optimizer

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

#logger = Logger('./logs')

In [10]:
# 开始训练

for epoch in range(num_epoches):

    print('epoch {}'.format(epoch + 1))

    print('*' * 10)

    running_loss = 0.0

    running_acc = 0.0

    for i, data in enumerate(train_loader, 1):

        img, label = data

        if use_gpu:

            img = img.cuda()

            label = label.cuda()

        img = Variable(img)

        label = Variable(label)

        # 向前传播

        out = model(img)

        loss = criterion(out, label)

        running_loss += loss.item() * label.size(0)

        _, pred = torch.max(out, 1)

        num_correct = (pred == label).sum()

        accuracy = (pred == label).float().mean()

        running_acc += num_correct.item()

        # 向后传播

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        # ========================= Log ======================

        step = epoch * len(train_loader) + i

        # (1) Log the scalar values

        info = {'loss': loss.item(), 'accuracy': accuracy.item()}
        if i % 300 == 0:

            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(

                epoch + 1, num_epoches, running_loss / (batch_size * i),

                running_acc / (batch_size * i)))

    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(

        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(

            train_dataset))))

    model.eval()

    eval_loss = 0

    eval_acc = 0

    for data in test_loader:

        img, label = data

        if use_gpu:

            img = Variable(img, volatile=True).cuda()

            label = Variable(label, volatile=True).cuda()

        else:

            img = Variable(img, volatile=True)

            label = Variable(label, volatile=True)

        out = model(img)

        loss = criterion(out, label)

        eval_loss += loss.item() * label.size(0)

        _, pred = torch.max(out, 1)

        num_correct = (pred == label).sum()

        eval_acc += num_correct.item()

    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(

        test_dataset)), eval_acc / (len(test_dataset))))

    print()

epoch 1
**********
[1/20] Loss: 2.293347, Acc: 0.171979
Finish 1 epoch, Loss: 2.283643, Acc: 0.245867


I:\Anaconda\lib\site-packages\ipykernel_launcher.py:90: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
I:\Anaconda\lib\site-packages\ipykernel_launcher.py:92: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Loss: 2.243141, Acc: 0.444800

epoch 2
**********
[2/20] Loss: 1.894836, Acc: 0.601849
Finish 2 epoch, Loss: 1.451135, Acc: 0.678717
Test Loss: 0.493695, Acc: 0.859800

epoch 3
**********
[3/20] Loss: 0.419506, Acc: 0.878177
Finish 3 epoch, Loss: 0.391792, Acc: 0.885717
Test Loss: 0.297122, Acc: 0.915000

epoch 4
**********
[4/20] Loss: 0.296837, Acc: 0.911875
Finish 4 epoch, Loss: 0.283179, Acc: 0.915817
Test Loss: 0.228981, Acc: 0.934400

epoch 5
**********
[5/20] Loss: 0.234756, Acc: 0.930990
Finish 5 epoch, Loss: 0.222118, Acc: 0.934233
Test Loss: 0.181566, Acc: 0.947100

epoch 6
**********
[6/20] Loss: 0.189863, Acc: 0.943880
Finish 6 epoch, Loss: 0.182535, Acc: 0.946150
Test Loss: 0.155599, Acc: 0.954100

epoch 7
**********
[7/20] Loss: 0.161407, Acc: 0.951589
Finish 7 epoch, Loss: 0.155281, Acc: 0.953600
Test Loss: 0.135904, Acc: 0.958900

epoch 8
**********
[8/20] Loss: 0.134943, Acc: 0.958411
Finish 8 epoch, Loss: 0.136724, Acc: 0.958333
Test Loss: 0.111428, Acc: 0.968700

In [11]:
# 保存模型

torch.save(model.state_dict(), './cnn.pth')